 # Web Scraping de forma Simples e Rápida


A extração, coleta e estruturação de dados em sites pode ser feita de um simples copiar e colar todo o código até extrair insights de importância para fins econômicos, financeiros e comerciais. 

Web Scraping torna esse processo autônomo, que é nosso propósito. Sendo uma maneira rápida e eficaz de coletar dados, que às vezes poderia ser impossível de acessar feito por uma pessoa.

Existe muitas bibliotecas Python para Web Scraping, podemos classificar as requisições em síncrona e assíncrona.

## Cenário
Sabe-se que notícias impactam o mercado financeiro. Alguns investidores querem ter a opção de não operar durante uma notícia, ou um determinado período dessa notícia. Principalmente quando se usa robôs de investimento, essa opção é muito bem-vinda.

Trataremos aqui de notícias pré-agendadas que tem hora, moeda ou par de moeda afetados, e a sua importância. No site https://br.investing.com/economic-calendar encontramos essa modalidade de notícias.

In [1]:
#Instalando bibliotecas
!pip install httpx bs4 dataclasses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 84 kB 329 kB/s 
     |████████████████████████████████| 68 kB 2.5 MB/s 
     |████████████████████████████████| 80 kB 4.3 MB/s 
     |████████████████████████████████| 58 kB 2.6 MB/s 


In [2]:
#Importando
import httpx
import asyncio
from dataclasses import dataclass, asdict
from bs4 import BeautifulSoup

## Optaremos por uma forma assíncrona

Somente para fins didáticos optaremos para a forma assíncona com resultado em formado dict. 

In [10]:
@dataclass
class Impacto:
    impactoBullish: int
    horarioArray: []
    origemMoeda: str


async with httpx.AsyncClient() as client:
        
    url = 'https://br.investing.com/economic-calendar/'
    
    response = await client.get(url)
    
    assert response.status_code == 200
    
    pagina = response.content
    informacoes = BeautifulSoup(pagina, 'lxml')
    tabela = informacoes.find('table', {'id': 'economicCalendarData'}).find('tbody').findAll('tr', {'class': 'js-event-item'})
    
    resultados = []
    for linha in tabela:
        novo_resultado = Impacto(
            impactoBullish = int(str(linha.find('td', {'class': 'sentiment'}).get('data-img_key')).replace('bull', '')),
            horarioArray = str(linha.get('data-event-datetime')).replace('/', '-')[-8:].split(':'),
            origemMoeda = linha.find('td', {'class': 'left flagCur noWrap'}).text.strip()
        )
                
        resultados.append(asdict(novo_resultado))

print(resultados) 

[{'impactoBullish': 2, 'horarioArray': ['03', '30', '00'], 'origemMoeda': 'EUR'}, {'impactoBullish': 1, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'GBP'}, {'impactoBullish': 3, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'GBP'}, {'impactoBullish': 3, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'GBP'}, {'impactoBullish': 2, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'GBP'}, {'impactoBullish': 2, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'GBP'}, {'impactoBullish': 2, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'EUR'}, {'impactoBullish': 3, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'EUR'}, {'impactoBullish': 1, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'EUR'}, {'impactoBullish': 1, 'horarioArray': ['04', '00', '00'], 'origemMoeda': 'EUR'}, {'impactoBullish': 1, 'horarioArray': ['05', '00', '00'], 'origemMoeda': 'CHF'}, {'impactoBullish': 1, 'horarioArray': ['06', '00', '00'], 'origemMoeda': 'EUR'}, {'impactoBullish': 1, 'hora

In [8]:
import pandas as pd

In [9]:
pd.DataFrame.from_dict(resultados)

,impactoBullish,horarioArray,origemMoeda
0,2,"[03, 30, 00]",EUR
1,1,"[04, 00, 00]",GBP
2,3,"[04, 00, 00]",GBP
3,3,"[04, 00, 00]",GBP
4,2,"[04, 00, 00]",GBP
...,...,...,...
63,1,"[20, 50, 00]",JPY
64,1,"[21, 00, 00]",NZD
65,1,"[21, 00, 00]",NZD
66,1,"[21, 00, 00]",NZD


##Conclusão
Podemos observar que o resultado é satisfatório, ao implementá-lo no código de um robô de forma simples e rápida como desejado.

A forma assíncrona ainda nos permite enviar várias solicitações, se for necessário, em páginas diferentes do site ao mesmo tempo, enquanto uma espera a resposta as outras já estão em solicitação, isso agiliza a consulta e extração de dados.

E com o parser 'lxml' nos dá uma rapidez maior ainda. No nosso exemplo foi uma simples consulta a uma tabela de uma única página. Mas isso poderia ser uma consulta a níveis exponenciais.

Outro ponto importante a salientar, podemos usar proxy nas consultas, no get do httpx. Qual também pode ser obtido em sites de proxy da mesma forma que foi do exemplo. Se torna um ciclo de uso do Web Scraping. Uma ótima ferramenta para os profissionais de dados.